In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder \
    .master("local[3]") \
    .appName("spark-hw") \
    .getOrCreate()

26/01/27 10:49:42 WARN Utils: Your hostname, MacBook-Air-Irina.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
26/01/27 10:49:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/27 10:49:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.5.1


In [4]:
actor_df = spark.read.csv('../data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('../data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('../data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('../data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('../data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('../data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('../data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('../data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('../data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('../data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('../data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('../data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('../data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('../data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
# тут має бути розвʼязок задачі

film_category_df \
    .join(category_df, on="category_id", how="inner") \
    .groupBy("name") \
    .agg(F.countDistinct("film_id").alias("film_count")) \
    .orderBy(F.col("film_count").desc()) \
    .show()


+-----------+----------+
|       name|film_count|
+-----------+----------+
|     Sports|        74|
|    Foreign|        73|
|     Family|        69|
|Documentary|        68|
|  Animation|        66|
|     Action|        64|
|        New|        63|
|      Drama|        62|
|      Games|        61|
|     Sci-Fi|        61|
|   Children|        60|
|     Comedy|        58|
|     Travel|        57|
|   Classics|        57|
|     Horror|        56|
|      Music|        51|
+-----------+----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [6]:
# тут має бути розвʼязок задачі

actor_df \
    .join(film_actor_df, on="actor_id", how="inner") \
    .join(inventory_df, on="film_id", how="inner") \
    .join(rental_df, on="inventory_id", how="inner") \
    .groupBy(
        "actor_id",
        "first_name",
        "last_name"
    ) \
    .agg(
        F.countDistinct("rental_id").alias("rental_count")
    ) \
    .orderBy(F.col("rental_count").desc()) \
    .limit(10) \
    .show()

+--------+----------+-----------+------------+
|actor_id|first_name|  last_name|rental_count|
+--------+----------+-----------+------------+
|     107|      GINA|  DEGENERES|         753|
|     181|   MATTHEW|     CARREY|         678|
|     198|      MARY|     KEITEL|         674|
|     144|    ANGELA|WITHERSPOON|         654|
|     102|    WALTER|       TORN|         640|
|      60|     HENRY|      BERRY|         612|
|     150|     JAYNE|      NOLTE|         611|
|      37|       VAL|     BOLGER|         605|
|      23|    SANDRA|     KILMER|         604|
|      90|      SEAN|    GUINESS|         599|
+--------+----------+-----------+------------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [7]:
# тут має бути розвʼязок задачі

category_spent_df = category_df \
    .join(film_category_df, on="category_id", how="inner") \
    .join(inventory_df, on="film_id", how="inner") \
    .join(rental_df, on="inventory_id", how="inner") \
    .join(payment_df, on="rental_id", how="inner") \
    .groupBy("category_id", "name") \
    .agg(F.round(F.sum("amount"), 2).alias("total_spent")) \
    .orderBy(F.col("total_spent").desc())

category_spent_df.limit(1).show()

+-----------+------+-----------+
|category_id|  name|total_spent|
+-----------+------+-----------+
|         15|Sports|    5314.21|
+-----------+------+-----------+



4.
Вивести назви фільмів, яких не має в inventory.

In [8]:
# тут має бути розвʼязок задачі

films_not_in_inventory_df = film_df.join(inventory_df, on="film_id", how="left_anti")
films_not_in_inventory_df.select("title").show()


+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [9]:
# тут має бути розвʼязок задачі

top_children_actors_df = actor_df \
    .join(film_actor_df, on="actor_id", how="inner") \
    .join(film_category_df, on="film_id", how="inner") \
    .join(category_df, on="category_id", how="inner") \
    .where(F.col("name") == "Children") \
    .groupBy("actor_id", "first_name", "last_name") \
    .agg(
        F.countDistinct("film_id").alias("children_film_count")
    ) \
    .orderBy(F.col("children_film_count").desc()) \
    .limit(3)

top_children_actors_df.show()


+--------+----------+---------+-------------------+
|actor_id|first_name|last_name|children_film_count|
+--------+----------+---------+-------------------+
|      17|     HELEN|   VOIGHT|                  7|
|     127|     KEVIN|  GARLAND|                  5|
|     140|    WHOOPI|     HURT|                  5|
+--------+----------+---------+-------------------+



Stop Spark session:

In [10]:
spark.stop()